In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Data Integration

Proses Data integration merupakan salah satu teknik data preprocessing dengan melakukan penggabungkan 2 file dataset peminjaman dengan us_state_code( kode negara domisili peminjam).

### Load Data

In [0]:
import pandas as pd
### Dataset lendingclub-issued-loans
data_lc_loan = pd.read_csv("/content/drive/My Drive/TA_01/Dataset/accepted_2007_to_2016.csv", delimiter=',', index_col=0)
data_lc_2016_2017 = pd.read_csv("/content/drive/My Drive/TA_01/Dataset/lc_2016_2017.csv", delimiter=',', index_col=0)
data_us_state_codes = pd.read_csv("/content/drive/My Drive/TA_01/Dataset/lendingclub-issued-loans/us-state-codes.csv", delimiter=',', index_col=0)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18,46,54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18,54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
data_loans = pd.concat([data_lc_loan, data_lc_2016_2017])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [0]:
data_loans2 = data_loans.reset_index()

In [0]:
data_loans2['state_code'] = data_loans2['addr_state']

In [0]:
data_loans_with_state = pd.merge(data_loans2, data_us_state_codes, on='state_code')

In [0]:
#data_loans_with_state.to_csv('/content/drive/My Drive/TA_01/Dataset/data_loans_with_state.csv')

In [0]:
# Define a function to visulize the features with missing values, and % of total values, & datatype
def missing_values_table(df):
     # Total missing values
    mis_val = df.isnull().sum()
    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_type = df.dtypes
    
    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent, mis_val_type], axis=1)
        
     # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values', 2: 'type'})
        
    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[ mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
        
    # Print some summary information
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n" "There are " + str(mis_val_table_ren_columns.shape[0]) + " columns that have missing values.")
        
    # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [0]:
missing_frac = data_loans_with_state.isnull().mean()
drop_list = sorted(missing_frac[missing_frac > 0.20].index)

In [0]:
def drop_cols(cols):
    data_loans_with_state.drop(labels=cols, axis=1, inplace=True)

In [0]:
drop_cols(drop_list)

In [0]:
from numpy.random import RandomState
rng = RandomState()
data_train = data_loans_with_state.sample(frac=0.75, random_state=rng)
data_test = data_loans_with_state.loc[~data_loans.index.isin(data_train.index)]

In [0]:
data_train

,id,acc_now_delinq,addr_state,annual_inc,application_type,collection_recovery_fee,collections_12_mths_ex_med,delinq_2yrs,dti,earliest_cr_line,emp_length,emp_title,funded_amnt,funded_amnt_inv,grade,home_ownership,initial_list_status,inq_last_6mths,installment,int_rate,issue_d,last_credit_pull_d,last_pymnt_amnt,last_pymnt_d,loan_amnt,loan_status,open_acc,out_prncp,out_prncp_inv,policy_code,pub_rec,purpose,pymnt_plan,recoveries,revol_bal,revol_util,sub_grade,term,title,tot_coll_amt,tot_cur_bal,total_acc,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp,total_rev_hi_lim,verification_status,zip_code,state_code,state
1129866,85759146,0.0,NY,139000.0,INDIVIDUAL,0.0,0.0,1.0,15.02,Oct-2005,8 years,det,20000.0,20000.0,C,RENT,w,1.0,688.33,14.49,Jul-2016,Jan-2017,688.33,Jan-2017,20000.0,Current,13.0,17711.24,17711.24,1.0,0.0,debt_consolidation,n,0.00,12880.0,57.0,C4,36 months,Debt consolidation,0.0,94445.0,36.0,3454.47,3454.47,1165.71,0.0,2288.76,22600.0,Verified,104xx,NY,NewYork
1626283,91654956,0.0,WA,24000.0,Individual,0.0,0.0,0.0,11.05,Jan-2005,3 years,dietary server,9500.0,9500.0,C,RENT,w,0.0,322.34,13.49,Oct-2016,Dec-2017,322.34,Sep-2017,9500.0,Late (31-120 days),3.0,6991.15,6991.15,1.0,0.0,credit_card,n,0.00,8593.0,44.8,C2,36 months,Credit card refinancing,0.0,8593.0,10.0,3531.50,3531.50,1022.65,0.0,2508.85,19200.0,Source Verified,993xx,WA,Washington
1211191,1349286,0.0,PA,54600.0,INDIVIDUAL,0.0,0.0,0.0,6.18,Jul-2002,10+ years,Church of the Living Christ,15000.0,15000.0,D,OWN,f,2.0,546.44,18.55,Jun-2012,Jan-2017,546.44,Jun-2013,15000.0,Charged Off,10.0,0.00,0.00,1.0,0.0,debt_consolidation,n,1.44,7398.0,62.0,D4,36 months,Debt Consolidation,NaN,NaN,31.0,6557.28,6557.28,2443.90,0.0,4111.94,NaN,Not Verified,151xx,PA,Pennsylvania
1648150,104341462,0.0,WI,60000.0,Individual,0.0,0.0,0.0,16.94,Sep-2003,10+ years,RN,6500.0,6500.0,A,MORTGAGE,w,0.0,200.68,6.99,Apr-2017,Dec-2017,200.68,Dec-2017,6500.0,Current,13.0,5170.59,5170.59,1.0,0.0,credit_card,n,0.00,12182.0,21.4,A2,36 months,Credit card refinancing,0.0,254786.0,28.0,1602.92,1602.92,273.51,0.0,1329.41,56800.0,Not Verified,530xx,WI,Wisconsin
345868,69803417,0.0,CA,35200.0,INDIVIDUAL,0.0,0.0,0.0,27.17,Oct-1990,10+ years,Exc Secretary,10425.0,10425.0,E,MORTGAGE,w,0.0,284.86,21.48,Jan-2016,Jan-2017,284.86,Dec-2016,10425.0,Current,9.0,9242.11,9242.11,1.0,2.0,debt_consolidation,n,0.00,21500.0,54.2,E5,60 months,Debt consolidation,0.0,28928.0,26.0,3121.02,3121.02,1938.13,0.0,1182.89,39700.0,Source Verified,936xx,CA,California
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143224,110019279,0.0,NY,90000.0,Individual,0.0,1.0,1.0,2.49,May-1997,2 years,Ironworker,9000.0,9000.0,C,RENT,w,0.0,305.81,13.59,Jun-2017,Dec-2017,305.81,Dec-2017,9000.0,Current,15.0,7741.53,7741.53,1.0,1.0,debt_consolidation,n,0.00,4937.0,39.5,C2,36 months,Debt consolidation,0.0,4937.0,36.0,1821.27,1821.27,562.80,0.0,1258.47,12500.0,Verified,104xx,NY,NewYork
1806596,83634339,0.0,MD,88100.0,INDIVIDUAL,0.0,0.0,2.0,13.83,Feb-1996,10+ years,Sub Contract Manager,15000.0,15000.0,B,MORTGAGE,w,0.0,487.47,10.49,Jul-2016,Jan-2017,487.47,Jan-2017,15000.0,Current,12.0,12814.66,12814.66,1.0,0.0,debt_consolidation,n,0.00,9506.0,60.5,B3,36 months,NaN,0.0,625511.0,22.0,2959.78,2959.78,774.44,0.0,2185.34,15700.0,Source Verified,211xx,MD,Maryland
743514,69593052,0.0,TX,108000.0,Individual,0.0,0.0,0.0,20.10,May-1997,2 years,Database Administrator,25000.0,25000.0,C,MORTGAGE,w,3.0,581.58,13.99,Jan-2016,Nov-2017,23001.09,Oct-2016,25000.0,Fully Paid,18.0,0.00,0.00,1.0,0.0,credit_card,n,0.00,21385.0,29.0,C4,60 months,Credit card refinancing,0.0,452877.0,32.0,27634.30,27634.30,2634.30,0.0,25000.00,74800.0,Verified,750xx,TX,Texas
1899243,75134911,0.0,SD,94320.0,INDIVIDUAL,0.0,0.0,0.0,18.73,Apr-1993,4 years,Sr. HR Generalist,35000.0,35000.0,C,MORTGAGE,w,0.0,1204.06

In [0]:
#data_train.to_csv('/content/drive/My Drive/TA_01/Dataset/data_train_loans.csv')

In [0]:
#data_test.to_csv('/content/drive/My Drive/TA_01/Dataset/data_test_loans.csv')